In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
index_url = 'https://en.wikisource.org/wiki/Portal:Constitutional_documents'

In [26]:
index_soup = BeautifulSoup(requests.get(index_url).text)
index_dict = {}

for li in index_soup.find_all('li')[4:-58]:
    for a in li.find_all('a'):
        if 'title' in a.attrs.keys():
            if 'page does not exist' not in a['title']:
                index_dict[a.text] = f"https://en.wikisource.org{a['href']}"

index_df = pd.DataFrame.from_dict(index_dict,orient='index').reset_index().rename(columns={'index':'constitution',0:'link'})

In [27]:
index_df

,constitution,link
0,Constitution of Afghanistan (1923),https://en.wikisource.org/wiki/Constitution_of...
1,Constitution of the Kingdom of Afghanistan,https://en.wikisource.org/wiki/Constitution_of...
2,Constitution of the Islamic Republic of Afghan...,https://en.wikisource.org/wiki/Constitution_of...
3,Constitution of Albania,https://en.wikisource.org/wiki/Constitution_of...
4,Constitution of Algeria,https://en.wikisource.org/wiki/Constitution_of...
...,...,...
479,Constitution of Yugoslavia (1992),https://en.wikisource.org/wiki/Constitution_of...
480,Rhodesian Declaration of Independence,https://en.wikisource.org/wiki/Unilateral_Decl...
481,Proposed Constitution of Zimbabwe,https://en.wikisource.org/wiki/Proposed_Consti...
482,Constitution of Liberland,https://en.wikisource.org/wiki/Constitution_of...


In [118]:
def get_text(url):
    try:
        soup = BeautifulSoup(requests.get(url).text)
        container = soup.find('div',{'class':'mw-parser-output'})
        if 'disambiguation' in soup.find('div', {'class':'mw-normal-catlinks'}).ul.text:
            links = []
            for li in container.find_all('li')[2:]:
                if 'page does not exist' not in li.a['title']:
                    links.append(f"https://en.wikisource.org{li.a['href']}")
            get_text(links[-1]) ## gets most recent
        else:
            if container.find('table') != None:
                container.find('table').decompose()
            if container.find('div',{'id':'toc'}) != None:
                container.find('div',{'id':'toc'}).decompose()
            return container.text.replace('[edit]', '')
    except:
        return None ## unknown error, @ln4 AttributeError: 'NoneType' object has no attribute 'ul' on iloc[261]
        ## unknown error, @ln2 ConnectionError: HTTPSConnectionPool(host='en.wikisource.orghttps', port=443): Max retries exceeded with url: //en.wikipedia.org/wiki/Basic_Laws_of_Israel

In [119]:
from tqdm import tqdm
tqdm.pandas()

index_df['text'] = index_df['link'].progress_apply(get_text)

100%|██████████| 484/484 [01:31<00:00,  5.26it/s]


In [123]:
index_df = index_df.drop(index_df.loc[index_df['text'] == ''].index).dropna().reset_index(drop=True)
index_df

,constitution,link,text
0,Constitution of Afghanistan (1923),https://en.wikisource.org/wiki/Constitution_of...,\nConstitution of Afghanistan (1923) related p...
1,Constitution of the Kingdom of Afghanistan,https://en.wikisource.org/wiki/Constitution_of...,\n\nConstitution of the Kingdom of Afghanistan...
2,Constitution of the Islamic Republic of Afghan...,https://en.wikisource.org/wiki/Constitution_of...,\n\nConstitution of the Islamic Republic of Af...
3,Constitution of Albania,https://en.wikisource.org/wiki/Constitution_of...,\n\nConstitution of Albania related portals: C...
4,Constitution of Algeria,https://en.wikisource.org/wiki/Constitution_of...,\n\nConstitution of Algeria (1996) related p...
...,...,...,...
440,Constitution of Yugoslavia (1974),https://en.wikisource.org/wiki/Constitution_of...,\n\n←Constitution of Yugoslavia (1963)\nConsti...
441,Constitution of Yugoslavia (1992),https://en.wikisource.org/wiki/Constitution_of...,\n←Constitution of Yugoslavia (1974)\nConstitu...
442,Proposed Constitution of Zimbabwe,https://en.wikisource.org/wiki/Proposed_Consti...,\nProposed Constitution of Zimbabwe (1999) r...
443,Constitution of Liberland,https://en.wikisource.org/wiki/Constitution_of...,\nThe Constitution of the Free Republic of Lib...


In [124]:
index_df.to_csv('constitutions.csv')